# Correr esta celda al iniciar (una sola vez)

In [8]:
import sys
import io
import os
import re
import time
import threading

import cv2
import numpy as np
from pyzbar.pyzbar import decode
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
from ipywidgets import Layout
from matplotlib import pyplot as plt

sys.path.insert(0, '../QR Query/Query QRs/Scripts')
sys.path.insert(0, '../FIND/Golden Master (AS IS)')
import qrQuery
import readImage as ri
from Browser import Browser
from DataBrowser import DataBrowser

# Widgets
queryOption = widgets.ToggleButtons(
    options=['Escribir QR', 'Seleccionar archivo', 'Rango QR'],
    description='Búsqueda:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
)
defaultAmount = widgets.ToggleButton(
    value=False,
    description='Seleccionar un número predeterminado de imágenes a mostrar?',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check',
    layout=Layout(width='50%', height='50px')
)
textQrs = widgets.Text(
    value='',
    placeholder='Escribe códigos QR válidos',
    description='Códigos QR:',
    disabled=False
)
uploads = widgets.FileUpload(
    accept='.jpg',
    multiple=True, # True to accept multiple files upload else False
    description='Imagen'
)
textRange = widgets.Text(
    value='',
    placeholder='Código de inicio y de final',
    description='Rangos QR:',
    disabled=False
)
saveImages = widgets.ToggleButton(
    value=False,
    description='Guardar imágenes mostradas?',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check',
    layout=Layout(width='50%', height='50px')
)

# Elige cómo hacer la búsqueda

In [9]:
display(queryOption)
display(defaultAmount)
display(saveImages)
print('Después de seleccionar, corre la siguiente celda')

ToggleButtons(button_style='info', description='Búsqueda:', options=('Escribir QR', 'Seleccionar archivo', 'Ra…

ToggleButton(value=False, button_style='info', description='Seleccionar un número predeterminado de imágenes a…

ToggleButton(value=False, button_style='info', description='Guardar imágenes mostradas?', icon='check', layout…

Después de seleccionar, corre la siguiente celda


# Escribe QRs o sube imágenes

In [10]:
if queryOption.value == 'Escribir QR':
    display(textQrs)
    print('Puedes introducir varios separando con coma, espacio o coma y espacio')
elif queryOption.value == 'Seleccionar archivo':
    display(uploads)
    print('Selecciona una o varias imágenes de tu computadora')
elif queryOption.value == 'Rango QR': 
    display(textRange)
    print('Puedes separar los códigos con coma, espacio o coma y espacio')
print('Luego de terminar, corre la siguiente celda')

Text(value='', description='Códigos QR:', placeholder='Escribe códigos QR válidos')

Puedes introducir varios separando con coma, espacio o coma y espacio
Luego de terminar, corre la siguiente celda


# Comenzar búsqueda

In [12]:
csvName = 'Query.csv'
csvFolder = './'
imagesFolder = './imagenes/'
qrList = []
if not os.path.isdir(csvFolder):
    os.mkdir(csvFolder)
if not os.path.isdir(imagesFolder):
    os.mkdir(imagesFolder)
csvFile = os.path.join(csvFolder, csvName)
print('Los datos completos de los QR están en el archivo {}'.format(csvFile))
if defaultAmount.value:
    amount = qrQuery.askForAmount('Máximo de imágenes a mostrar? Presiona enter al final ')
if queryOption.value == 'Escribir QR':
    qrList = qrQuery.splitInputQrs(textQrs.value)
    if not qrList:
        print('El espacio de escritura está vacío')
elif queryOption.value == 'Rango QR':
    inputList = qr_query.splitInputQrs(textRange.value)
    if inputList:
        if len(inputList) == 2:
            if inputList[0] > inputList[1]:
                start = int(inputList[1])
                finish = int(inputList[0])
            else:
                start = int(inputList[0])
                finish = int(inputList[1])
            qrList = np.arange(start, finish+1, 1)
            qrList = [str(qr) for qr in qrList]
        else:
            print('Escribe un QR de inicio y uno de final\n')
    else:
        print('El espacio de escritura está vacío')
elif queryOption.value == 'Seleccionar archivo':
    for upload in uploads.metadata:
        fileName = upload['name']
        imageBytes = uploads.value[upload['name']]['content']
        imageBytes = io.BytesIO(imageBytes)
        image = Image.open(imageBytes)
        qr = str(decode(image)[0][0])[2:-1]
        qrList.append(qr)
    if len(qrList) == 0:
        print('Ningún QR pudo ser detectado\n')
    elif len(qrList) < len(uploads.metadata):
        print('No todos los QR pudieron detectarse\n')

for qr in qrList:
    qr = qrQuery.fixQr(qr)
    dataExistsWidget = widgets.Valid(
    value=False,
    description='Registros',
    )
    imagesExistsWidget = widgets.Valid(
        value=False,
        description='Imágenes',
    )
    validQrWidget = widgets.Valid(
        value=False,
        description='QR válido',
    )
    print('\nQR a analizar: {}'.format(qr))
    if qrQuery.isValidQr(qr):
        validQrWidget.value = True
        # DATA GATHERING
        data = qrQuery.getData(qr)
        if not qrQuery.isNone(data):
            dataExistsWidget.value = True
            data.to_csv(csvFile, mode='a', header=False, index=False) 
        # IMAGES GATHERING
        if 'amount' in locals(): 
            if amount > 0:
                images = queryQr.getImages(qr)
                nImages = len(images)
                if nImages > 0:    
                    imagesExistsWidget.value = True
                    ri.showImages(images[0:amount])
                    if saveImages.value:
                        for image in images[0:amount]:
                            imageName = '{}{}_{}.png'.format(imagesFolder, qr, str(image['count']))
                            plt.imsave(imageName, cv2.cvtColor(image['image'], cv2.COLOR_BGR2RGB))
                display(imagesExistsWidget)
            else:
                pass
        elif 'amount' not in locals():
            images = qrQuery.getImages(qr)
            nImages = len(images)
            if nImages > 0:  
                imagesExistsWidget.value = True
                prompt = 'Existen {} fotos, cuántas quieres ver? '.format(nImages)
                amount = qrQuery.askForAmount(prompt)
                ri.showImages(images[0:amount])
                del amount
                if saveImages.value:
                    for image in images[0:amount]:
                        imageName = '{}{}_{}.png'.format(imagesFolder, qr, str(image['count']))
                        plt.imsave(imageName, cv2.cvtColor(image['image'], cv2.COLOR_BGR2RGB))
            display(imagesExistsWidget)
    else:
        validQrWidget.value = False
        print('El QR: {} es inválido'.format(qr))
        continue
    display(validQrWidget)
    display(dataExistsWidget)
    print('\n\n')
if 'amount' in locals():
    del amount
print('Búsqueda terminada')

Los datos completos de los QR están en el archivo ./Query.csv
Máximo de imágenes a mostrar? Presiona enter al final 0

QR a analizar: 102190800201058


,No. Imagen,Validez,Fecha + 5hrs,Ciudad,Fecha Consulta,Base,QR,ESAT6,CFP10,RV1681,Control
0,0,valid,2019-10-16 15:43,Ndola,2019-10-16T15:49,CLOUD,102190800201058,Positive,Negative,Negative,Negative


Valid(value=True, description='QR válido')

Valid(value=True, description='Registros')




Búsqueda terminada
